In [217]:
import numpy as np
import matplotlib.pyplot as plt
import skimage
from skimage import io, filters, transform
import os
import json
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

# Training data

In [218]:
#go to folder with 500 images
os.chdir('train500')
X_train = np.array([io.imread(str(i)+'.jpg') for i in range(1,501)])
os.chdir('..')

In [85]:
train_json = json.load(open('train.json'))

In [225]:
y_train = []
for i in range(1,501):
    labels = train_json['annotations'][i]['labelId']
    labels = np.array(list(map(int,labels)))
    y_train.append(labels)
y_train = np.array(y_train)

In [227]:
#pad labels with zeros
y_train_pad = []

for i in y_train:
    num_zeros = 16 - len(i)
    zeros = [0] * num_zeros
    zeros = np.array(zeros)
    new = np.append(i,zeros)
    y_train_pad.append(new)
y_train_pad = np.array(y_train_pad)

Resize images to fit (200,128,3) and flatten the resized images

In [228]:
print('smallest vertical:',min(i.shape[0] for i in X_train))
print('smallest horizontal:',min(i.shape[1] for i in X_train))

smallest vertical: 200
smallest horizontal: 128


In [229]:
#All images resized to the smallest dimensions
X_train_resized = [transform.resize(img,(200,128,3)) for img in X_train]
X_train_flat = np.array([img.flatten() for img in X_train_resized])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


# Validation data

In [230]:
os.chdir('val500')
X_test = np.array([io.imread(str(i)+'.jpg') for i in range(1,501)])
os.chdir('..')

In [231]:
X_test_resized = [transform.resize(img,(200,128,3)) for img in X_test]
X_test_flat = np.array([img.flatten() for img in X_test_resized])

/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [232]:
val_json = json.load(open('validation.json'))

In [233]:
y_test = []
for i in range(1,501):
    labels = val_json['annotations'][i]['labelId']
    labels = np.array(list(map(int,labels)))
    y_test.append(labels)
y_test = np.array(y_test)

#pad labels with zeros
y_test_pad = []
for i in y_test:
    num_zeros = 16 - len(i)
    zeros = [0] * num_zeros
    zeros = np.array(zeros)
    new = np.append(i,zeros)
    y_test_pad.append(new)
y_test_pad = np.array(y_test_pad)

# KNN

In [234]:
knn = KNeighborsClassifier()
knn.fit(X_train_flat,y_train_pad)
pred = knn.predict(X_test_flat)

In [238]:
pred[0]

array([ 52, 106, 153,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0])

In [239]:
y_test_pad[0]

array([  95.,   17.,   66.,  214.,  164.,  137.,   20.,  204.,  184.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.])

In [235]:
#from sklearn.metrics import accuracy_score
#acc = accuracy_score(y_test_pad, pred)

In [236]:
#Hamming loss https://stackoverflow.com/questions/38697982/python-scikit-learn-multi-class-multi-label-performance-metrics
acc = np.sum(np.not_equal(y_test_pad, pred))/float(y_test_pad.size)
acc


0.45700000000000002